# Probe Trajectory Estimation on Atlas

## Modes
### Mode 1
**TrajectoryPlanning(id_insert, id_target, x_insert_mm, y_insert_mm, x_target_mm, y_target_mm)**

The mode 1 estimates a trajectory based on predefined start and end of desired trajectory. The input variables "id_insert/target" is the plate number (identical to file name) of insertion/target point. x/y_insert/target_mm are the x/y-coordinates of the start/end points on atlas.

### Mode 2
**TrajectoryPlanning2(id_insert, id_target, x_insert_mm, y_insert_mm, x_target_mm, y_target_mm, trajectory_length)**

The mode 2 estimates a trajectory based on two structures and trajectory length. One structure should be the insertion point, given as id_insert and x/y_insert_mm. Another structure can be any region on atlas.

## Mode 1

In [76]:
# Specify the file ID on atlas
id_insert = 44
id_target = 87

# Specify the insertion and target coordinates in mm
x_insert_mm = 4
y_insert_mm = 1.6

x_target_mm = 5.7
y_target_mm = 7.4

# Function
TrajectoryPlanning(id_insert, id_target, x_insert_mm, y_insert_mm, x_target_mm, y_target_mm)

Insertion: Plate 44 , sinus 6.61 (mm)
Target: Plate 87 , sinus 0.71 (mm)
Pitch: 46.63 (deg)
Azimuth: 73.93 (deg)
90-azimuth: 16.07 (deg)
Trajectory Length: 8.45 (mm)


## Mode 2

In [6]:
# Specify the file ID on atlas
id_insert = 40
id_target = 87

# Specify the insertion and target coordinates in mm
x_insert_mm = 3.5
y_insert_mm = 1.2

x_target_mm = 5.7
y_target_mm = 7.4

trajectory_length = 9

# Function
TrajectoryPlanning2(id_insert, id_target, x_insert_mm, y_insert_mm, x_target_mm, y_target_mm, trajectory_length)

Insertion: Plate 40 , sinus 7.18 (mm)
Target: Plate 87 , sinus 0.71 (mm)
Estimated pitch: 47.78 (deg)
Estimated azimuth: 71.22 (deg)
90-azimuth: 18.78 (deg)


## Function

In [4]:
# Import libraries
from PIL import Image, ImageDraw, ImageFont
import math
import os 
import sys
from datetime import datetime
import glob
import numpy as np
import cv2
import pickle
import matplotlib.pyplot as plt

def TrajectoryPlanning(id_insert, id_target, x_insert_mm, y_insert_mm, x_target_mm, y_target_mm):
    # Coordinates in pixels
    x_lim = [755, 210] # [0,8]
    y_lim = [200, 993] # [0, 11.6]

    # Convert coordinates
    x_insert = x_lim[0] + x_insert_mm * (x_lim[1] - x_lim[0])/8
    y_insert = y_lim[0] + y_insert_mm * (y_lim[1] - y_lim[0])/11.6
    x_target = x_lim[0] + x_target_mm * (x_lim[1] - x_lim[0])/8
    y_target = y_lim[0] + y_target_mm * (y_lim[1] - y_lim[0])/11.6

    # AP coordinates for slices
    with open("AP", "rb") as fp:   # Unpickling
        AP_ls = pickle.load(fp)

    # AP coordinates of insertion and target
    AP_insert = AP_ls[id_insert-1]
    AP_target = AP_ls[id_target-1]

    # AP coordinates for slices
    with open("Plate", "rb") as fp:   # Unpickling
        plt_ls = pickle.load(fp)

    # Plate number of insertion and target
    plt_insert = plt_ls[id_insert-1]
    plt_target = plt_ls[id_target-1]

    # load images and mark trajectories
    atlas_path = 'atlas'
    img_ls = [f'{i+1:04d}.jpg' for i in range(len(os.listdir(atlas_path)))]

    # Make save directory
    now = datetime.now()
    dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
    os.mkdir(dt_string)
    #print(AP_insert, AP_target)

    # Compute and print the angles

    p3d_insert = np.array([x_insert_mm, y_insert_mm, AP_insert])
    p3d_target = np.array([x_target_mm, y_target_mm, AP_target])
    total_trajectory_dist = np.linalg.norm(p3d_insert - p3d_target)

    # ML_vector = np.array([y_target_mm - y_insert_mm, x_target_mm - x_insert_mm])
    rot_AP_vector = np.array([AP_target - AP_insert, x_target_mm - x_insert_mm])
    # AP_angle = 90 - math.degrees(math.acos(np.linalg.norm(rot_AP_vector) / total_trajectory_dist))
    # ML_angle = math.degrees(math.acos(abs(ML_vector[0]) / math.sqrt(ML_vector[0]**2 + ML_vector[1]**2)))

    transverse_angle = math.degrees(math.acos(abs(y_target_mm - y_insert_mm)/total_trajectory_dist))
    azimuth_angle = 90 - math.degrees(math.atan2(abs(rot_AP_vector[1]),abs(rot_AP_vector[0])))

    for i in range(id_insert,id_target+1):
        im_path = os.path.join(atlas_path,img_ls[i-1])
        im = np.array(Image.open(im_path))
        AP = AP_ls[i-1]
        # Compute x,y of trajectory in the current slice
        x = x_target + (x_insert - x_target) * (AP - AP_target) / (AP_insert - AP_target)
        y = y_target + (y_insert - y_target) * (AP - AP_target) / (AP_insert - AP_target)

        AP_dist_mm = np.abs(AP_insert - AP)
        target_AP_dist_mm = np.abs(AP_insert - AP_target)


        if(target_AP_dist_mm > 0):
            probe_depth = (AP_dist_mm / target_AP_dist_mm) * total_trajectory_dist
        else:
            probe_depth = total_trajectory_dist

        font = cv2.FONT_HERSHEY_SIMPLEX
        im = cv2.putText(im, f'{probe_depth} mm', (np.array(im).shape[1]//2 - 50, 100), font, 
                   1, [0,0,255], 2, cv2.LINE_AA)

        im = cv2.circle(im, np.array([x,y]).astype(np.int16), 6, [255,0,0], 2)


        # Save images
        cv2.imwrite(os.path.join(dt_string, 'AP' + str("{:.2f}".format(AP))) + '.jpg', im)

    # Print
    print("Insertion: Plate", str(plt_insert), ", sinus", AP_insert, "(mm)")
    print("Target: Plate", str(plt_target), ", sinus", AP_target, "(mm)")

    # print("AP_angle:", round(AP_angle,2), "degree")
    # print("ML_angle:", round(ML_angle,2), "degree")
    print("Pitch:", round(transverse_angle,2), "(deg)")
    print("Azimuth:", round(azimuth_angle,2), "(deg)")
    print("90-azimuth:", round(90 - azimuth_angle,2), "(deg)")

    print(f'Trajectory Length: {round(total_trajectory_dist,2)} (mm)')

In [5]:
# Import libraries
from PIL import Image, ImageDraw, ImageFont
import math
import os 
import sys
from datetime import datetime
import glob
import numpy as np
import cv2
import pickle
import matplotlib.pyplot as plt

def TrajectoryPlanning2(id_insert, id_target, x_insert_mm, y_insert_mm, x_target_mm, y_target_mm, trajectory_length):
    # Coordinates in pixels
    x_lim = [755, 210] # [0,8]
    y_lim = [200, 993] # [0, 11.6]

    # Convert coordinates
    x_insert = x_lim[0] + x_insert_mm * (x_lim[1] - x_lim[0])/8
    y_insert = y_lim[0] + y_insert_mm * (y_lim[1] - y_lim[0])/11.6
    x_target = x_lim[0] + x_target_mm * (x_lim[1] - x_lim[0])/8
    y_target = y_lim[0] + y_target_mm * (y_lim[1] - y_lim[0])/11.6

    # AP coordinates for slices
    with open("AP", "rb") as fp:   # Unpickling
        AP_ls = pickle.load(fp)

    # AP coordinates of insertion and target
    AP_insert = AP_ls[id_insert-1]
    AP_target = AP_ls[id_target-1]

    # AP coordinates for slices
    with open("Plate", "rb") as fp:   # Unpickling
        plt_ls = pickle.load(fp)

    # Plate number of insertion and target
    plt_insert = plt_ls[id_insert-1]
    plt_target = plt_ls[id_target-1]

    # load images and mark trajectories
    atlas_path = 'atlas'
    img_ls = [f'{i+1:04d}.jpg' for i in range(len(os.listdir(atlas_path)))]

    # Compute and print the angles

    p3d_insert = np.array([x_insert_mm, y_insert_mm, AP_insert])
    p3d_target = np.array([x_target_mm, y_target_mm, AP_target])
    trajectory_dist = np.linalg.norm(p3d_insert - p3d_target)

    # ML_vector = np.array([y_target_mm - y_insert_mm, x_target_mm - x_insert_mm])
    rot_AP_vector = np.array([AP_target - AP_insert, x_target_mm - x_insert_mm])
    # AP_angle = 90 - math.degrees(math.acos(np.linalg.norm(rot_AP_vector) / total_trajectory_dist))
    # ML_angle = math.degrees(math.acos(abs(ML_vector[0]) / math.sqrt(ML_vector[0]**2 + ML_vector[1]**2)))

    transverse_angle = math.degrees(math.acos(abs(y_target_mm - y_insert_mm)/trajectory_dist))
    azimuth_angle = 90 - math.degrees(math.atan2(abs(rot_AP_vector[1]),abs(rot_AP_vector[0])))

    # Make save directory
    now = datetime.now()
    dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
    os.mkdir(dt_string)

    i = id_insert
    probe_depth = 0
    while probe_depth < trajectory_length and i in range(1,88):
        im_path = os.path.join(atlas_path,img_ls[i-1])
        im = np.array(Image.open(im_path))
        AP = AP_ls[i-1]

        # Compute x,y of trajectory in the current slice
        x = x_target + (x_insert - x_target) * (AP - AP_target) / (AP_insert - AP_target)
        y = y_target + (y_insert - y_target) * (AP - AP_target) / (AP_insert - AP_target)

        AP_dist_mm = np.abs(AP_insert - AP)
        target_AP_dist_mm = np.abs(AP_insert - AP_target)

        if(target_AP_dist_mm > 0):
            probe_depth = (AP_dist_mm / target_AP_dist_mm) * trajectory_dist
        else:
            probe_depth = trajectory_length

        font = cv2.FONT_HERSHEY_SIMPLEX
        im = cv2.putText(im, f'{probe_depth} mm', (np.array(im).shape[1]//2 - 50, 100), font, 
                   1, [0,0,255], 2, cv2.LINE_AA)

        im = cv2.circle(im, np.array([x,y]).astype(np.int16), 6, [255,0,0], 2)


        # Save images
        cv2.imwrite(os.path.join(dt_string, 'AP' + str("{:.2f}".format(AP))) + '.jpg', im)

        if id_insert < id_target:
            i += 1
        elif id_insert > id_target:
            i -= 1

    # Print
    print("Insertion: Plate", str(plt_insert), ", sinus", AP_insert, "(mm)")
    print("Target: Plate", str(plt_target), ", sinus", AP_target, "(mm)")

    # print("AP_angle:", round(AP_angle,2), "degree")
    # print("ML_angle:", round(ML_angle,2), "degree")
    print("Estimated pitch:", round(transverse_angle,2), "(deg)")
    print("Estimated azimuth:", round(azimuth_angle,2), "(deg)")
    print("90-azimuth:", round(90 - azimuth_angle,2), "(deg)")